# SWOT

This notebook demonstrates access to SWOT level 2 data. Broad information about the dataset can be found in the PODAAC website (see [here](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_NALT_GDR_2.0))


**Requirements to run this notebook**
1. Have an Earth Data Login account
2. Have a Bearer Token.


**Objectives**

To demonstrate a workflow for remote access and plotting of Complex (**Level 2** with Groups)  SWOT Data via OPeNDAP


`Author`: Miguel Jimenez-Urias, '24

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from pydap.client import open_url
import json
import cartopy.crs as ccrs

### Access EARTHDATA

The access link can be found at [PODACC](https://podaac.jpl.nasa.gov/dataset/SWOT_L2_NALT_GDR_2.0). This may require to be logged on to EarthDataLogin. There is data for 2023 and 2024.  



In [ ]:
data_url1 = 'https://opendap.earthdata.nasa.gov/collections/C2799438313-POCLOUD/granules/SWOT_GPR_2PfP507_010_20230501_003247_20230501_012352'

### Add to session's headers Token Authorization


In [ ]:
edl_token = "YourToken"

auth_hdr="Bearer " + edl_token

In [ ]:
# pass Token Authorization to a new Session.

my_session = requests.Session()
my_session.headers={"Authorization": auth_hdr}

**Create a dataset access via pydap**




In [ ]:
dataset1 = open_url(data_url1, session=my_session, protocol="dap4")

In [ ]:
dataset1.tree()

```{note}
PyDAP accesses the remote dataset's metadata, and no data has been downloaded yet!
```

**This is a dataset pointing to a remote data location**


Data remains remote, no data has been downloaded.

In [ ]:
dataset1['data_01/mean_dynamic_topography'].shape

In [ ]:
dataset1['data_01/time'].shape

In [ ]:
print('total array memory: ', dataset1.nbytes/1e9)

**Inspect the values**


- `longitude`
- `latitude`
- `time`



In [ ]:
dataset1['data_01/time'].attributes

In [ ]:
%%time
dyn_topo = dataset1['data_01/mean_dynamic_topography'][:] # downloads as BaseType - a thin wrapper for numpy arrays

**Maps**

Refers to the coverage of the Satellite track. This is, how the trajectory "**maps**" with `time` (i.e. the `dimension`)

In [ ]:
dyn_topo.Maps

In [ ]:
longitude1 = dataset1[dyn_topo.Maps[0]][:]
latitude1 = dataset1[dyn_topo.Maps[1]][:]

In [ ]:
longitude1.attributes

In [ ]:
latitude1.attributes

**Decoding data values**

`xarray` decodes time and spatial values internally by default, everytime one accesses the data values, whereas currently there is no such method within `pydap` to do so. But it is often useful to understand how this works internally, and what type of parameters are used for decoding. Because OPeNDAP is based on the NetCDF data model, it if a CF-compliant software. Below are some of the most used metadata attributes associated for decoding data:

**CF - Conventions**

In OPeNDAP's metadata rich datasets, each contains standard attributes used to describe missing data, units in which the data is presented, and any stretching/scaling of the values. 

- `standard name`
- `units`
- `_FillValue`
- `scale_factor`
- `off_set`


In [ ]:
def decode(variable) -> np.ndarray:
    """Decodes the variable BaseType according with atributes:
        _FillValue
        scale_factor
    """
    scale_factor = 1
    _Fill_value = None

    if 'scale_factor' in variable.attributes:
        scale_factor = variable.scale_factor
    if '_FillValue' in variable.attributes:
        data = np.where(variable.data == variable._FillValue, np.nan, variable.data)    
    else:
        data = variable.data
    return scale_factor * data

**Lets make some plots!**

`OPeNDAP` does NOT include a plotting service, but `OPeNDAP`-served data integrates easily with plotting packages like
- `Matplotlib`
- `Cartopy`


In [ ]:
plt.figure(figsize=(15, 5))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()
ax.stock_img() # comment this line if you do not want any background color
plt.scatter(x=decode(longitude1), y=decode(latitude1), c=decode(dyn_topo), marker='.',  cmap='jet')
plt.colorbar().set_label(dyn_topo.name + ' ['+dyn_topo.units+']')

**Fig. 1** Global map showing the track (trajectory) of the sampled (satellite) data. Values refer to mean dynamic topography. 

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(decode(dyn_topo), 'k', lw=3)
plt.ylabel(dyn_topo.name + ' ['+dyn_topo.units+']', fontsize=15)
plt.xlabel('Along Track Samples', fontsize=15)
plt.show()

**Fig 2.** Along track values. The first values along the track represent the North Atlantic, whereas the steep dropoff in dynamic topography represent the Southern Ocean.